# Visualizing NURBS DIC Results from Synthetic Testing
## Sam Potter
## Current: 3/6/19

## Path and Imports

In [1]:
import sys
import os

sys.path.extend(['C:\\Users\\potterst1\\Desktop\Repositories\BitBucket\dic',
                 'C:/Users/potterst1/Desktop/Repositories/BitBucket/dic'])
sys.path.extend(['/workspace/stpotter/git/bitbucket/dic'])
from dic import visualize
import cv2
from matplotlib import pyplot as plt
import numpy as np
from geomdl import BSpline as bs
from geomdl import utilities as gutil
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import stats

## Magics

In [2]:
%matplotlib inline

## Read in results data from text files

In [16]:
# Define a function for ease of use. Brittle and expects a certain format, but okay for now

def results_open(fname):
    
    f = open(fname, 'r')
    lines = f.readlines()
    lines = lines[3:]
    data = np.zeros((len(lines), 4))
    for i in range(0, len(data)):
        entry = list(map(float, lines[i].split()))
        data[i, :] = np.array(entry)
    return data
        
# Get synthetic
fname = 'C:\\Users\\potterst1\\Desktop\\Repositories\\BitBucket\\dic\\scripts\\3Steps\\stretchx\\01synthetic.txt'
synth01 = results_open(fname)

fname = 'C:\\Users\\potterst1\\Desktop\\Repositories\\BitBucket\\dic\\scripts\\3Steps\\stretchx\\12synthetic.txt'
synth12 = results_open(fname)

In [17]:
synth01

array([], shape=(0, 4), dtype=float64)

## Visualize Applied Synthetic Displacement, Deformation

## Visualize Minimization Result Displacement, Deformation

## Compute Differences between Applied and Minimized Results

## Visualize Differences betwteen Applied and Minimized Results 

## Compute Statistics on Differences between Applied and Minimized Results